# Testes com modelos decoders-only para geração automática de questões

runtime type: t4 gpu

In [ ]:
# Instalação das bibliotecas
!pip install transformers accelerate torch
# Tranformers -> Carrega modelos pré-treinados e realiza inferências neles (otimizada para GPU)
# Accelerate -> Otimiza hardware. Alocação automatica das partes do modelo (device_map="auto")
# Torch -> PyTorch. Backend do modelo. Cria e manipula tensores (torch_dtype=torch.float16)

In [ ]:
# Carregando modelo usando a biblioteca Transformers:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# AutoModelForCausalLM - > carrega modelos para tarefas de Causal Language Modeling
  # Causal language model -> prevê próxima palavra com base no contexto anterior
# AutoTokenizer -> Automatiza carregamento de tokenizers

#modelo_name = "eleutherAI/gpt-j-6b"
model_name = "facebook/opt-6.7b"

In [ ]:
# Carregando tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

print(model.half())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 4096, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
      (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-31): 32 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
            (out_proj): Linear(in_features=4096, out_features=4096, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=4096, out_features=16384, bias=True)
          (fc2): Linear(in_features=16384, out_features=4096, bias=True)
          (final_layer_norm): 

In [ ]:
# Função de exemplo para geração de questão
def gerar_questao(prompt):
  inputs = tokenizer(prompt, return_tensors="pt").to("cuda") # transforma entrada em tensor
    # return_tensors="pt" -> Indica que retorna tensores pyTorch
    # to("cuda") -> transferi tensores para GPU (se disponível) acelerando processamento

  outputs = model.generate(inputs.input_ids, max_length=200, num_return_sequences=1, temperature=0.7, do_sample=True, top_k=50)
    # geração do texto
    # inputs.input_ids -> tensor de IDs de tokens gerados pelo tokenizer. Ponto de partida para geração
    # max_length -> tamanho máximo da saída em tokens
    # do_sample=True-> Ativa amostragem em vez de escolher sempre saída com maior probabilidae
      # Permite saídas mais criativas e variadas
    # top_k=50 -> k palavras com maior probabilidade podem ser escolhidas
      # Melhor que pegar a palavra mais provável para saídas mais variadas

  return tokenizer.decode(outputs[0], skip_special_tokens=True) # decode (voltando para texto legível)
    # outputs[0] -> primeira sequência gerada
    # skip_special_tokens -> Remove tokens especiais como os de início e fim de sequência

In [ ]:
# Exemplo de geração de pergunta
prompt = "Crie uma questão teórica sobre a sintaxe básica do Java, explicando como se escreve uma classe."
questao = gerar_questao(prompt)
print(questao)

Crie uma questão teórica sobre a sintaxe básica do Java, explicando como se escreve uma classe.                   Aí você faz o exame com os dados suficientes e você consegue acessar o curso.
Obrigado pela dica.


In [ ]:
# Exemplo de geração de pergunta
prompt = """Por favor, gere uma questão teórica de Java sobre a sintaxe básica de como escrever uma classe. A questão deve ser no seguinte formato:

Pergunta: Qual é a estrutura básica de uma classe em Java?
a) Declaração de variáveis, depois métodos, e por fim a palavra-chave 'public'
b) Declaração da classe, variáveis e métodos
c) Método principal, depois declarações de variáveis e classes internas
d) Somente declarações de métodos

Resposta correta: b
"""
questao = gerar_questao(prompt)
print(questao)

Por favor, gere uma questão teórica de Java sobre a sintaxe básica de como escrever uma classe. A questão deve ser no seguinte formato:

Pergunta: Qual é a estrutura básica de uma classe em Java?
a) Declaração de variáveis, depois métodos, e por fim a palavra-chave 'public'
b) Declaração da classe, variáveis e métodos
c) Método principal, depois declarações de variáveis e classes internas
d) Somente declarações de métodos

Resposta correta: b

Fonte:

Custo:

Durante a tarde é possí
